In [ ]:
!pip install langchain langchain-community langchain-openai openai tiktoken sentence-transformers faiss-cpu rank_bm25 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.8 MB/s eta 0:00:00


In [ ]:
import os
from getpass import getpass

In [ ]:
from langchain_openai import ChatOpenAI

# os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0)

Введите ваш OpenAI API ключ··········


<!-- <center> <img src='../images/RAG.png' width="628" height="428" align="right">
 -->

# Document loader & Text Splitter

In [ ]:
import json
import pandas as pd
js_path = 'all-articles.JSON'

# Load the JSON file
with open(js_path, 'r') as file:
    data = json.load(file)

In [ ]:
from langchain.document_loaders import CSVLoader, DataFrameLoader

In [ ]:
# Определим лоадер
df = pd.read_json(js_path)
loader = DataFrameLoader(df, page_content_column="article_text")
documents = loader.load()

In [ ]:
df.head(1)

,article_id,article_text,article_slug,article_title,article_published_at
0,244,"Иногда отель — это больше, чем просто место, г...",10-rossijskih-otelej-s-istoriej,10 российских отелей с историей,2023-08-18 07:02:58.675000+00:00


##  Text splitters - разбиваем текст на фрагменты




In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=10000,
    chunk_overlap=100,
    length_function=len,
)
split_documents = splitter.split_documents(documents)
len(split_documents)

1937

# Embedding models  


In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings_api_model = OpenAIEmbeddings()

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

hf_embeddings_model = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru", model_kwargs={"device": "cuda"}
)

<ipython-input-13-313f6cca95b3>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/521k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(
    split_documents, hf_embeddings_model
)

db.save_local("faiss_db")

In [ ]:
retriever = db.as_retriever(
    search_type="similarity",  # тип поиска похожих документов
    k=4,  # количество возвращаемых документов (Default: 4)
    score_threshold=None,  # минимальный порог для поиска "similarity_score_threshold"
)

#Retriever + LLM

In [ ]:
from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Создаём простой шаблон
template = """

your role is to be a travel guide. i will tell you my location and you will recommend a place to visit near me. Sometimes, i will also specify the kind of places i like to visit. you will also give me some suggestions for similar places that are close to my first destination. In the answer use big and bold letters in titles and points and numbers so it looks good and is easy to read and understand, offer to articulate the places you provided, or add more places. don't mention this is a game.
Answer the question based only on the following context:

{context}

Question: {question}
"""
# Создаём промпт из шаблона
prompt = ChatPromptTemplate.from_template(template)


# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


# Создаём цепочку
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke("Куда съездить и где отдохнуть в Москве?")

'**Рекомендация:**\n\n### **Парк Горького и Воробьёвы горы**\n1. **Парк Горького:** Прогуляйтесь по парку, наслаждаясь атмосферой и красотой природы. Здесь можно арендовать велосипед и покататься по аллеям.\n2. **Воробьёвы горы:** Посетите смотровую площадку для великолепного вид на город. Прокатитесь по окрестностям и насладитесь панорамой Москвы.\n\n### **Набережные**\n1. **Нагатинская набережная — Красная площадь:** Прокатитесь вдоль Москвы-реки, наслаждаясь видами и атмосферой города.\n2. **Большой Кремлёвский сквер — Кремлёвская набережная:** Посетите исторические места и насладитесь прогулкой вдоль реки.\n\n### **Бульварное кольцо**\n1. **Прокатитесь по бульварному кольцу:** Насладитесь атмосферой и красотой центра Москвы, гуляя по бульварам и улицам.\n\n### **Садовое кольцо**\n1. **Исследуйте Садовое кольцо:** Прогуляйтесь по улицам и насладитесь архитектурой и историей города.\n\n### **ВДНХ и Ботанический сад**\n1. **Посетите ВДНХ и Ботанический сад:** Отправьтесь на прогулку п